In [1]:
import numpy as np
import pandas as pd
import string
import calendar
import re
import math
import sympy
import matplotlib.pyplot as plt
import pprint

from fractions import Fraction
from scipy import stats
from scipy import special


print(f"numpy version is {np.__version__}")
print(f"pandas version is {pd.__version__}")


numpy version is 1.20.3
pandas version is 1.3.2


In [2]:
# def getDiscreteProbsExpon(input_ranges):
#     prob_out = []
#     for i in input_ranges:
#         if i[0] == '<':
#            high = float(i[1:])
#             prob_out.append(stats.expon.cdf(high, scale ))
#         elif i[0] == '>':
#             low = float(i[1:])
#             prob_out.append(1 - stats.expon.cdf(low, scale ))
#         else:
#             low, high = i.split('-')
#             prob_out.append(stats.expon.cdf(float(high), scale ) - stats.expon.cdf(float(low), scale ))
#     return np.array(prob_out)

# def getDiscreteProbsPoisson(input_ranges):
#     prob_out = []
#     for i in input_ranges:
#         if i[0] == '<':
#             high = float(i[1:])
#             prob_out.append(stats.poisson.cdf(high))
#         elif i[0] == '>':
#             low = float(i[1:])
#             prob_out.append(1 - stats.poisson.cdf(low))
#         elif len(i.split('-')) == 2:
#             low, high = i.split('-')
#             prob_out.append(stats.poisson.cdf(float(high)) - stats.poisson.cdf(float(low)))
#         else:
#             prob_out.append(stats.poisson.pmf(float(i)))
#     return np.array(prob_out)

def getDiscreteProbs_PMF(in_dist, input_ranges):
    prob_out = []
    for i in input_ranges:
        if i[0] == '<':
            high = float(i[1:])
            prob_out.append(in_dist.cdf(high))
        elif i[0] == '>':
            low = float(i[1:])
            prob_out.append(1 - in_dist.cdf(low))
        elif len(i.split('-')) == 2:
            low, high = i.split('-')
            prob_out.append(in_dist.cdf(float(high)) - in_dist.cdf(float(low)))
        else:
            prob_out.append(in_dist.pmf(float(i)))
    return np.array(prob_out)


def getDiscreteProbs_PDF(in_dist, input_ranges):
    prob_out = []
    for i in input_ranges:
        
        if i[0] == '<':
            high = float(i[1:])
            prob_out.append(in_dist.cdf(high))
            
        elif i[0] == '>':
            low = float(i[1:])
            prob_out.append(1 - in_dist.cdf(low))
            
        elif len(i.split('-')) == 2:
            low, high = i.split('-')
            prob_out.append(in_dist.cdf(float(high)) - in_dist.cdf(float(low)))
            
    return np.array(prob_out)

### Monte Carlo simulation for p-value

In [28]:
def getPVal_MCSIM(prob_in, nRuns, sampleSize):
    ani_rng             = np.random.default_rng()
    all_TS = []
    for i in range(nRuns):
        oneSample       = ani_rng.choice(prob.shape[0], p = prob_in, size = (sampleSize, 1))
        a, b            = np.unique(oneSample, return_counts = True)
        testStat        = -sampleSize + np.divide((b ** 2) , sampleSize * prob_in).sum()
        all_TS.append(testStat)
    return all_TS

10000


### Goodness of fit test with all parameters specified

In [72]:
# manual data entry

# prob        = np.array([1/4, 1/2, 1/4])
# names       = ['White', 'Pink', "Red"]
# X           = np.array([141, 291, 132])

# read data from text file
# two-column data

rawData     = np.loadtxt('GOF_test_specified.txt', dtype = ('str', 'str'), delimiter= ' ')

# four-column data
# rawData     = np.loadtxt('GOF_test_specified.txt', delimiter= ' ')
# rawData     = np.sort(rawData.reshape((-1, 2)), axis = 0)

names       = np.squeeze(rawData[:, 0]).astype(str)
X           = np.squeeze(rawData[:, 1]).astype(int)



# probabilities of the assumed distribution function

# Uniform
# prob        = (np.ones(X.shape[0]) * 1/X.shape[0]).round(4)

# Poisson
# poisMean = 0.3
# prob = getDiscreteProbsPoisson(names, poisMean)

# Exponential
# exponMean   = 50
# prob        = getDiscreteProbsExpon(names, exponMean)

# discrete probaiblity distribution
ani_prob    = stats.poisson(mu = 4)
prob        = getDiscreteProbs_PMF(ani_prob, names)

# continuous probaiblity distribution

# ani_prob    = stats.norm(loc = 100, scale = 15)
# prob        = getDiscreteProbs_PDF(ani_prob, names)

sig         = 5
n           = X.sum()
k           = prob.shape[0]

testStat    = -n + np.divide((X ** 2) , n * prob).sum()

# chi-squared approximation to find p value
pVal        = 1 - stats.chi2.cdf(testStat, k - 1)

# MC simulations to find p-value
# mc_runs     = 100000
# sample_Size = n
# all_TS      = np.array(getPVal_MCSIM(prob, mc_runs, sample_Size))
# n_smaller   = all_TS[all_TS > testStat]
# pVal        = n_smaller.shape[0] / all_TS.shape[0]

hypo        = 'rejected' if pVal < (0.01 * sig) else 'accepted'


#### output as table

In [73]:
outDict     ={'Test Statistic'              : '{:.2e}'.format(testStat),
            '$p$ value \%'                  : '{:.2f}'.format(100 * pVal),
            'Significance ($\\alpha$) \%'   : '{:.2f}'.format(sig),
            'null hypothesis ($H_0$)'       : hypo,
            'minimum $n p_i$'               : '{:.0f}'.format(min(np.multiply(prob, n)))
            }
dfb = pd.DataFrame.from_dict(outDict, orient='index', columns = ['Value'])

df                      = pd.DataFrame(np.vstack((X.astype(str), prob.round(4))).T, columns= ['$X_i$', '$p_i$'], index=names)
df                      = df.append(pd.Series({'$X_i$' : n, '$p_i$' : 1}, name = 'Total'))

In [74]:
s3          = df.to_latex(escape=False, column_format='@{}lrr@{}')
s3          = s3.replace('Total', '\midrule\nTotal')
s3          = s3.replace('{} &', 'Grade &')
s4          = dfb.to_latex(escape=False, column_format='@{}lr@{}')
s4          = s4.replace('{} &     Value', '\multicolumn{2}{c}{\\texttt{Goodness of Fit Test}}')

print('\\begin{table}[H]')
print('\\centering')
print('\\begin{minipage}{0.4\\textwidth}')
print('\\centering')
print(s3)
print('\\end{minipage}')
print('\\begin{minipage}{0.4\\textwidth}')
print('\\centering')
print(s4)
print('\\end{minipage}')
print('\\end{table}')
print('\\bigskip')


\begin{table}[H]
\centering
\begin{minipage}{0.4\textwidth}
\centering
\begin{tabular}{@{}lrr@{}}
\toprule
Grade & $X_i$ &   $p_i$ \\
\midrule
0     &     0 &  0.0183 \\
1     &     5 &  0.0733 \\
2     &    22 &  0.1465 \\
3     &    23 &  0.1954 \\
4     &    32 &  0.1954 \\
5     &    22 &  0.1563 \\
6     &    19 &  0.1042 \\
7     &    13 &  0.0595 \\
8     &     6 &  0.0298 \\
9     &     4 &  0.0132 \\
10    &     4 &  0.0053 \\
11    &     0 &  0.0019 \\
\midrule
Total &   150 &       1 \\
\bottomrule
\end{tabular}

\end{minipage}
\begin{minipage}{0.4\textwidth}
\centering
\begin{tabular}{@{}lr@{}}
\toprule
\multicolumn{2}{c}{\texttt{Goodness of Fit Test}} \\
\midrule
Test Statistic             &  2.62e+01 \\
$p$ value \%               &      0.60 \\
Significance ($\alpha$) \% &      5.00 \\
null hypothesis ($H_0$)    &  rejected \\
minimum $n p_i$            &         0 \\
\bottomrule
\end{tabular}

\end{minipage}
\end{table}
\bigskip


### Contingency tables test of independence

In [2]:
sig                 = 5

conTab              = pd.read_csv('Con_table.txt', sep=',', index_col=0)
r, s                = conTab.shape[0], conTab.shape[1]
N_ij                = conTab.copy()

print(conTab)

conTab['Row Sum']   = conTab.sum(axis = 1)
conTab.loc['Total'] = conTab.sum()
n                   = int(conTab['Row Sum'][-1])

p                   = conTab['Row Sum'][:-1] / n
q_series            = conTab.loc['Total']
q                   = q_series[:-1] / q_series[-1]

p_arr               = np.tile(p, (s, 1)).T
q_arr               = np.tile(q, (r, 1))

T_con               = -n + (np.divide(N_ij ** 2, n * np.multiply(p_arr, q_arr))).to_numpy().sum()
p_con               = 1 - stats.chi2.cdf(T_con, (r-1)*(s-1))
hypo                = 'rejected' if p_con < (0.01 * sig) else 'accepted'


                  Observations   Rain
Sky Color                            
Red                         61     26
Mainly red                 194     52
Yellow                     159     81
Mainly yellow              188     86
Red and yellow             194     52
Gray                       302    167


#### output contingency table and summary

In [3]:
outDict     ={'X levels ($r$)'              : '{:d}'.format(r),
            'Y levels ($s$)'                : '{:d}'.format(s),
            'Data points ($n$)'             : '{:d}'.format(n),
            'Test Statistic'                : '{:.2e}'.format(T_con),
            '$p$ value \%'                  : '{:.2f}'.format(100 * p_con),
            'Significance ($\\alpha$) \%'   : '{:.2f}'.format(sig),
            'null hypothesis ($H_0$)'       : hypo
            }
dfc = pd.DataFrame.from_dict(outDict, orient='index', columns = ['Value'])

# df                      = pd.DataFrame(np.vstack((X.astype(str), prob.round(4))).T, columns= ['$X_i$', '$p_i$'], index=names)
# df                      = df.append(pd.Series({'$X_i$' : n, '$p_i$' : 1}, name = 'Total'))



s6          = dfc.to_latex(escape=False, column_format='@{}lr@{}')
s6          = s6.replace('{} &     Value', '\multicolumn{2}{c}{\\texttt{Contingency Table Independence}}')

s5          = conTab.to_latex()
s5          = s5.replace('Total', '\midrule\nTotal')


print('\\begin{table}[H]')
print('\\centering')
print(s5)
print('\\end{table}')

print('\\bigskip')

print('\\begin{table}[H]')
print('\\centering')
print(s6)
print('\\end{table}')

print('\\bigskip')

\begin{table}[H]
\centering
\begin{tabular}{lrrr}
\toprule
{} &   Observations &   Rain &  Row Sum \\
Sky Color       &                &        &          \\
\midrule
Red             &             61 &     26 &       87 \\
Mainly red      &            194 &     52 &      246 \\
Yellow          &            159 &     81 &      240 \\
Mainly yellow   &            188 &     86 &      274 \\
Red and yellow  &            194 &     52 &      246 \\
Gray            &            302 &    167 &      469 \\
\midrule
Total           &           1098 &    464 &     1562 \\
\bottomrule
\end{tabular}

\end{table}
\bigskip
\begin{table}[H]
\centering
\begin{tabular}{@{}lr@{}}
\toprule
\multicolumn{2}{c}{\texttt{Contingency Table Independence}} \\
\midrule
X levels ($r$)             &         6 \\
Y levels ($s$)             &         2 \\
Data points ($n$)          &      1562 \\
Test Statistic             &  2.74e+01 \\
$p$ value \%               &      0.00 \\
Significance ($\alpha$) \% &      5.00 

### Kolmogorov-Smirnov test

In [15]:
raw             = np.sort(np.loadtxt('KS_test.txt', delimiter=','))

def empirical_CDF(x_in, obs_in):
    return ((obs_in <= x_in).sum()) / obs_in.shape[0]

0.9166666666666666

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [22]:
norm_ex     = stats.norm(loc = 3, scale = 4)
stats.kstest((np.log(raw) - 3) / 4, 'norm')

KstestResult(statistic=0.43163234117636107, pvalue=0.015081277352786637)

181